In [17]:
import pandas as pd
import numpy
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import TensorBoard


In [18]:
data = pd.read_csv('/content/Sentiment.csv')
data = data[['text', 'sentiment']]

In [19]:
# all string to lowercase Read more about lambda() "https://realpython.com/python-lambda/"
data['text'] = data['text'].apply(lambda x: x.lower())
# using regular expression preprocess the text by removing everything that is not [a-zA-z0-9\s]
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

print(data[data['sentiment'] == 'Positive'].size)
print(data[data['sentiment'] == 'Negative'].size)
print(data[data['sentiment'] == 'Neutral'].size)

data.head()

4472
16986
6284


,text,sentiment
0,rt nancyleegrahn how did everyone feel about t...,Neutral
1,rt scottwalker didnt catch the full gopdebate ...,Positive
2,rt tjmshow no mention of tamir rice and the go...,Neutral
3,rt robgeorge that carly fiorina is trending h...,Positive
4,rt danscavino gopdebate w realdonaldtrump deli...,Positive


In [20]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, maxlen=28)

embed_dim = 128
lstm_out = 196

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.25, random_state=42)

#build the model
batch_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

tb = TensorBoard(log_dir="logs/{}", histogram_freq=0, write_graph=True, write_images=True)
fmodel = model.fit(X_train, Y_train, epochs=5, batch_size=batch_size, verbose=2, callbacks=[tb])

Epoch 1/5
82/82 - 28s - loss: 0.8848 - accuracy: 0.6182
Epoch 2/5
82/82 - 24s - loss: 0.7288 - accuracy: 0.6861
Epoch 3/5
82/82 - 24s - loss: 0.6591 - accuracy: 0.7178
Epoch 4/5
82/82 - 24s - loss: 0.6223 - accuracy: 0.7357
Epoch 5/5
82/82 - 24s - loss: 0.5945 - accuracy: 0.7506


In [21]:
#save the model
model.save('/content/saved_model.h5')
m = load_model('/content/saved_model.h5')

text = [['A lot of good things are happening. We are respected again throughout the world, and thats a great thing. @realDonaldTrump']]
df = pd.DataFrame(text, index=range(0, 1, 1), columns=list('t'))
df['t'] = df['t'].apply(lambda x: x.lower())
df['t'] = df['t'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [22]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['t'].values)
X = tokenizer.texts_to_sequences(df['t'].values)
# Pads sequences to the same length.
X = pad_sequences(X, maxlen=28)

In [23]:
#predect it
output = m.predict(X)
print('Output:', output)
print(numpy.where(max(output[0])), ":", (max(output[0])))
print(numpy.argmax(output))
print(model.summary())

Output: [[0.82746    0.05978538 0.11275464]]
(array([0]),) : 0.82746
0
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None
